In [1]:
import random
import pandas as pd
import numpy as np

In [2]:
dataInflu = pd.read_csv("influencers.csv",)
dataFinal = dataInflu.values
dataInflu.head()

,id,followerCount,engagementRate
0,1,6207,1.0
1,2,38237,5.8
2,3,14840,0.6
3,4,43851,1.4
4,5,15530,3.5


<img src="followersFuzzy.png"/>

In [3]:
def clasificationFoll(foll):
    resultFoll = []
    #bit range
    if 0 <= foll <= 33000:
        resultFoll.append(['bit',1])
    elif 33000 <= foll <= 33500:
        resultFoll.append(['bit',-(foll-33500)/(33500-33000)]) #down
        
    #medium range    
    if 33000 <= foll <= 33500:
        resultFoll.append(['medium',(foll-33000)/(33500-33000)]) #up
    elif 33500 <= foll <= 66500:
        resultFoll.append(['medium',1])
    elif 66500 <= foll <= 67000:
        resultFoll.append(['medium',-(foll-67000)/(67000-66500)]) #down
        
    #big range    
    if 66500 <= foll <= 67000:
        resultFoll.append(['big',(foll-66500)/(67000-66500)]) #up
    elif 67000 <= foll <= 100000:
        resultFoll.append([ 'big',1])
    return resultFoll

In [4]:
# clasificationFoll(38237)

<img src="rateFuzzy.png"/>

In [5]:
def clasificationRate(rate):
    resultRate = []
#     bad range
    if 0 <= rate <= 3:
        resultRate.append(['bad',1])
    elif 3 <= rate <= 3.5:
        resultRate.append(['bad',-(rate-3.5)/(3.5-3)]) #down  
        
    #normal range
    if 3 <= rate <= 3.5:
        resultRate.append(['normal',(rate-3)/(3.5-3)]) #up
    elif 3.5 <= rate <= 6.5:
        resultRate.append(['normal',1])
    elif 6.5 <= rate <= 7:
        resultRate.append(['normal',-(rate-7)/(7-6.5)]) #down
        
    #good range  
    if 6.5 <= rate <= 7:
        resultRate.append(['good',(rate-6.5)/(7-6.5)]) #up
    elif 7 <= rate <= 10:
        resultRate.append(['good',1])
    return resultRate

In [6]:
# clasificationRate(5.8)

<img src="truthTable.png"/>

In [7]:
def truthTable(fuzzy1,fuzzy2):
    if (fuzzy1=='bit') and (fuzzy2=='bad'):
        answer = 'low'
    elif (fuzzy1=='bit') and (fuzzy2=='normal'):
        answer = 'low'
    elif (fuzzy1=='bit') and (fuzzy2=='good'):
        answer = 'strong'
    elif (fuzzy1=='medium') and (fuzzy2=='bad'):
        answer = 'low'
    elif (fuzzy1=='medium') and (fuzzy2=='normal'):
        answer = 'low'
    elif (fuzzy1=='medium') and (fuzzy2=='good'):
        answer = 'strong'
    elif (fuzzy1=='big') and (fuzzy2=='bad'):
        answer = 'low'
    elif (fuzzy1=='big') and (fuzzy2=='normal'):
        answer = 'strong'
    elif(fuzzy1=='big') and (fuzzy2=='good'):
        answer = 'strong'
    else:
        answer = 'unknow!'
    return answer

In [8]:
def inference(fuzzy1,fuzzy2):
    result = []
        
    if fuzzy1[0][1] < fuzzy2[0][1]:
        result.append([fuzzy1[0][1],truthTable(fuzzy1[0][0],fuzzy2[0][0])])
    else:
        result.append([fuzzy2[0][1],truthTable(fuzzy1[0][0],fuzzy2[0][0])])
        
    if len(fuzzy2) > 1:
        if fuzzy1[0][1] < fuzzy2[1][1]:
            result.append([fuzzy1[0][1],truthTable(fuzzy1[0][0],fuzzy2[1][0])])
        else:
            result.append([fuzzy2[1][1],truthTable(fuzzy1[0][0],fuzzy2[1][0])])
    
    if len(fuzzy1) > 1:
        if fuzzy1[1][1] < fuzzy2[0][1]:
            result.append([fuzzy1[1][1],truthTable(fuzzy1[1][0],fuzzy2[0][0])])
        else:
            result.append([fuzzy2[0][1],truthTable(fuzzy1[1][0],fuzzy2[0][0])])

    if len(fuzzy1) > 1 and len(fuzzy2) >1 :
        if fuzzy1[1][1] < fuzzy2[1][1]:
            result.append([fuzzy1[1][1],truthTable(fuzzy1[1][0],fuzzy2[1][0])])
        else:
            result.append([fuzzy2[1][1],truthTable(fuzzy1[1][0],fuzzy2[1][0])])
#     print(result)    
    #selection
    dataTes = result
        
    arrTemp = []
    strongTemp = [-1,'unknow']
    truthTemp = []
    lowTemp = [-1,'unknow']
    pLow = 0 #pointer
    cLow = 0 #counter

    pStrong = 0 #pointer
    cStrong = 0 #counter

    for i in range(len(dataTes)):
        if i==0:
            truthTemp = [-1,'unknow']
    #     print('nilai awal truth ke ',i)
    #     print()
        for j in range(len(dataTes)):
            j = j+1
    #         print('nilai j =',j)
    #         print('test ',dataTes[i][0], truthTemp[0],'--',dataTes[i][1], truthTemp[1])
            if dataTes[i][0] > truthTemp[0] or dataTes[i][1] != truthTemp[1]:
    #             print('masuk')
                if dataTes[i][1] == 'low':
                    if dataTes[i][0]>dataTes[pLow][0] or cLow==0:
                        lowTemp = dataTes[i]
                        truthTemp = dataTes[i]
                        pLow = i
                        cLow = cLow+1

    #                     print('lowTemp = ',lowTemp)

                elif dataTes[i][1] == 'strong':
    #                 print(dataTes[i][0],'>', dataTes[pStrong][0]) 
                    if dataTes[i][0]>dataTes[pStrong][0] or cStrong==0:
                        strongTemp = dataTes[i]
                        truthTemp = dataTes[i]
                        pStrong = i
                        cStrong = cStrong+1
    #                     print('strongTemp = ',strongTemp)
    if lowTemp[1]!='unknow':
        arrTemp.append(lowTemp)
    if strongTemp[1]!='unknow':
        arrTemp.append(strongTemp)

#     len(dataTes)
#     print(lowTemp,strongTemp)
    arrTemp
    return arrTemp

<img src="defuzzy.png"/>

In [9]:
def clasificationDefuzzy(deff):
    resultDeff = []
#     low range
    if 0 <= deff <= 45:
        resultDeff.append(1)
    elif 45 <= deff <= 55:
        resultDeff.append(-(deff-55)/(55-45)) #down   
    
    if 0<= deff <= 44:
        resultDeff.append(0)
        
    #strong range  
    if 45 <= deff <= 55:
        resultDeff.append((deff-45)/(55-45)) #up
    elif 55 <= deff <= 100:
        resultDeff.append(0)
        resultDeff.append(1)
    return resultDeff

In [10]:
def defuzzyfication(resultInference):
    arr = []
    index = []
    arrTemp = []
    arrFinal = []
    for i in range(7,100,4):
        arr.append(clasificationDefuzzy(i))
        index.append(i)

    for i in range(len(arr)):
    #     print(arr[i][0],' >' ,resultInference[0][0])
        if arr[i][0] > resultInference[0][0]:
            arr[i][0] = resultInference[0][0]
        
        if len(resultInference) > 1:
            if arr[i][1] > resultInference[1][0]:
                arr[i][1] = resultInference[1][0]
    #         print(index[i], arr[i])
        if arr[i][0] >= arr[i][1]:
            arrTemp.append(arr[i][0])
        else:
            arrTemp.append(arr[i][1])
        resTemp = arrTemp[i]*index[i]
        arrFinal.append(resTemp)

#     print(arrFinal)
    a = np.sum(arrTemp)
    b = np.sum(arrFinal)
    result = b/a
#     print(result)
    return result

In [11]:
# print(defuzzyfication(resultInference))

In [12]:
#main Program
follCount = []
rate = []
result = []
record = []
for i in range (len(dataFinal)):
#     print(dataFinal[i][1], ' - ',dataFinal[i][2])
    follCount.append(dataFinal[i][1])
    rate.append(dataFinal[i][2])
    
    fuzzy1 = clasificationFoll(follCount[i])
    fuzzy2 = clasificationRate(rate[i])
    resultInference = inference(fuzzy1,fuzzy2)
    
    resultDefuzzy = defuzzyfication(resultInference)
#     print(round(resultDefuzzy))
    result.append([resultDefuzzy,i])

result.sort(reverse = True)
for i in range(len(result)):
#     print(result[i][1])
    record.append(result[i][1])
print(record[0:20])
pd.DataFrame(record[0:20]).to_csv("chosen.csv")

[90, 79, 75, 52, 35, 12, 5, 99, 94, 66, 50, 26, 10, 7, 61, 98, 97, 96, 95, 93]
